## Lab5
netid: yxw190015

name: Yile Wang

In [3]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from numpy import zeros, dot, array
import pickle
import matplotlib.pyplot as plt
import json
import string
import time

## Modularity

The first function below calculates modularity for *directed* networks and also returns the maximum modularity value $Q_{\text{max}}$ (NetworkX's modularity function does not report the $Q_{\text{max}}$ value). The second function calculates scalar assortativity (NetworkX's assortativity functions differ from our book definition). 

In [4]:
def modularity(G,c):
    d = dict()
    for k,v in enumerate(c):
        for n in v:
            d[n] = k
    L = 0
    for u,v,data in G.edges.data():
        L += data['weight']
    Q, Qmax = 0,1
    for u in G.nodes():
        for v in G.nodes():
            if d[u] == d[v]:
                Auv = 0
                if G.has_edge(v,u):
                    Auv = G[v][u]['weight']
                Q += ( Auv - G.in_degree(u,weight='weight')*G.out_degree(v,weight='weight')/L )/L
                Qmax -= ( G.in_degree(u,weight='weight')*G.out_degree(v,weight='weight')/L )/L
    return Q, Qmax

def scalar_assortativity(G,d):
    x = zeros(G.number_of_nodes())
    for i,n in enumerate(G.nodes()):
        x[i] = d[n]

    A = array(nx.adjacency_matrix(G).todense().T)
    M = 2*A.sum().sum()
    ki = A.sum(axis=1) #row sum is in-degree
    ko = A.sum(axis=0) #column sum is out-degree
    mu = ( dot(ki,x)+dot(ko,x) )/M

    R, Rmax = 0, 0
    for i in range(G.number_of_nodes()):
        for j in range(G.number_of_nodes()):
             R += ( A[i,j]*(x[i]-mu)*(x[j]-mu) )/M
             Rmax += ( A[i,j]*(x[i]-mu)**2 )/M

    return R, Rmax

In [5]:
G = nx.read_weighted_edgelist('fifa1998.edgelist',create_using=nx.DiGraph)

c = {
    'group1': {'Argentina','Brazil','Chile','Mexico','Colombia','Jamaica','Paraguay'},
    'group2': {'Japan','SouthKorea'},
    'group3': {'UnitedStates'},
    'group4': {'Nigeria','Morocco','SouthAfrica','Cameroon','Tunisia','Iran','Turkey'},
    'group5': {'Scotland','Belgium','Austria','Germany','Denmark','Spain','France','GreatBritain','Greece','Netherlands','Norway','Portugal','Italy','Yugoslavia','Romania','Bulgaria','Croatia','Switzerland'}
    }
Q, Qmax = modularity(G,c.values())
print('FIFA exports by geographic region is assortatively mixed: %1.4f/%1.4f' % (Q,Qmax))

c = {
    'exporters': {'Argentina','Brazil','Chile','Colombia','Mexico','Scotland','Belgium','Austria','Denmark','France','Greece','Netherlands','Portugal','Yugoslavia','Croatia','Jamaica','Cameroon','Nigeria','Morocco','Tunisia'},
    'importers': {'Paraguay','SouthKorea','UnitedStates','SouthAfrica','Iran','Turkey','Germany','Spain','GreatBritain','Norway','Italy','Romania','Bulgaria','Switzerland','Japan'}
    }
Q, Qmax = modularity(G,c.values())
print('FIFA exports by importers/exporters is disassortatively mixed: %1.4f/%1.4f' % (Q,Qmax))

FIFA exports by geographic region is assortatively mixed: 0.1200/0.5505
FIFA exports by importers/exporters is disassortatively mixed: -0.0185/0.5748


### Describing why I see the values
Firstly I need to define what's the meaning of `Q` and `Qmax`. The `Q` means the modularity, whose definition is that the fraction of edges that connect nodes of the same type minus fraction of edges if it's random graph. If the `Q` is higher, it means that it is more likely an assortative mixing in the graph (More organized). The `Qmax` represents an ideal situation that the modularity score of every edge is between nodes of same type. 

In the FIFA network analysis, I can know that if we group players by geographic region,the `Q` is larger than 0, which means that there is higher fraction that similar type of nodes are connected. The real life situation is that there are more players trading within the same continent. 

For the second group, importers and exporters network, the modularity score `Q` is negative, which means that the edges between nodes are not likely coming from the same type of nodes, which makes sense here because the condition to make connections between nodes is that player from country A is playing at country B now. In the importers/exporters network, there may have fewer connections between nodes both in the importers/exporters type (There are more connections/edges between importers and exporters!). That's the reason why it has negative modularity scores.

For the similar `Qmax`, it is because that the network is still the same network with same nodes and edges. The only thing different is the $\delta(c_i, c_j)$, which means all the edges between nodes of same type. By grouping nodes in different way, there might be slightly different value for $\delta(c_i, c_j)$, but overall the two `Qmax` should be similar.

## Assortativity

In [6]:
gdp = pickle.load(open('gdp.pkl','rb'))
life_expectancy = pickle.load(open('life_expectancy.pkl','rb'))
tariff = pickle.load(open('tariff.pkl','rb'))

G = nx.read_weighted_edgelist('world_trade_2014.edgelist',create_using=nx.DiGraph)

R, Rmax = scalar_assortativity(G,gdp)
print('Assortativity by GDP: %1.4f' % (R/Rmax))
R, Rmax =  scalar_assortativity(G,life_expectancy)
print('Assortativity by life expectancy: %1.4f' % (R/Rmax))
R, Rmax =  scalar_assortativity(G,tariff)
print('Assortativity by tariff: %1.4f' % (R/Rmax))

Assortativity by GDP: -0.0005
Assortativity by life expectancy: 0.1281
Assortativity by tariff: 0.1191


### Describe world trading system and reason
First we need to know what the edges mean in the network. From the definition of scalar assortativity, it measures how much two variables vary together. In the world trading network, the edge is defined as the trading value between two countries. 

The scalar assortativity is to describe the covariance between nodes $x_i$ and $x_j$. In simple words, it captures how much $x_i$ node is different from the mean. The $\mu$ is the mean, which is the weighted average of scalar node value, proportioned to their degree. The concept of the `R/Rmax` is similar with `Q/Qmax`.

From the assortativity statistics, we can know that GDP has almost no salient effect to the trading behaviors between two countries. The `R/Rmax` of the GDP is close to random graph, which means that `GDP` of a country is not an important factor to affect the trading relationship. A potential reason is that GDP doesn't mean the average wealth level for individuals. For example, Taiwan and Japanese may not have large number in `GDP` but the citizens are wealth than many of countries with high `GDP`. 

However, the life expectancy and tariff could be important factors, which make senses too. To some extent, the life expectancy can reflect the aging population distribution in the country. The increase of the life expectancy can also reflect the development of economic system and medical system for a country. These can be an important indicator for the value of goods they trades, then influencing the total trade values. Also, countries with similar aging population may have similar demand and requirment for most of the people, which could be an factor for the positive assortativity value here; The tariff is another important factor for trading. An equal (similiar level) of the tariff between two countries can be easier to reach an equilibrium then trading more goods to each other. If the difference of tariff between two countries are huge (I taxed you 10% and you taxed me 50%), there are less likely to have a stable and large amount of trading between two countries.

### algebraic activity to show the following simplification

The steps are here:

$$
\frac{1}{2m}\sum_{i=1}^{n}\sum_{j=1}^{n} A_{ij}(x_i - \mu)(x_j - \mu) = \frac{1}{2m}\sum_{i=1}^{n}\sum_{j=1}^{n} A_{ij}(x_ix_j - \mu x_i - \mu x_j + \mu ^2) 

$$
The covariance equation of the $x_i$ and $x_j$ can be re-written as:
$$
cov(x_i, x_j) = \mathbb{E}(x_i x_j) - \mu{_{x_i}} \mathbb{E}(x_j) - \mu{_{x_j}}\mathbb{E}(x_i) + \mu{_{x_i}} \mu{_{x_j}}
$$

Then

$$
cov(x_i, x_j) =\mathbb{E}(x_i x_j) - \mu{_{x_i}} \mu{_{x_j}} - \mu{_{x_j}} \mu{_{x_i}} + \mu{_{x_i}} \mu{_{x_j}} 
$$

We got:

$$
cov(x_i, x_j) =\mathbb{E}(x_i x_j) - \mu{_{x_i}} \mu{_{x_j}}  
$$

Finally the equation is

$$
cov(x_i, x_j) =\mathbb{E}(x_i x_j) - \mathbb{E}(x_i) \mathbb{E}(x_j)
$$

Then, the original equation is

expected value of the $(x_i x_j)$ is 
$$
\mathbb{E}(x_i x_j) = \frac{1}{2m}\sum_{i=1}^{n}\sum_{j=1}^{n} A_{ij}(x_i x_j) 
$$

The expected value of the $x_i$ and $x_j$ is $\mu$, then we can multiply them to $\mu{^2}$

$$
\mathbb{E}(x_i)(x_j) = \frac{1}{4m^2}\sum_{i=1}^{n} k_i x_i \sum_{j=1}^{n} k_j x_j = \mu{^2}  
$$

Thus, we can know that the 

$$
R=\frac{1}{2m}\sum_{i=1}^n \sum_{j=1}^n A_{ij}(\mathbb{E}[x_i x_j] - \mathbb{E}[x_i]\mathbb{E}[x_j]) =\frac{1}{2m}\sum_{i=1}^n \sum_{j=1}^n A_{ij}x_ix_j-\frac{1}{4m^2}\sum_{i=1}^{n} k_i x_i \sum_{j=1}^{n} k_j x_j = \frac{1}{2m}\sum_{i=1}^n \sum_{j=1}^n A_{ij}x_ix_j-\mu{^2} 
$$

## Community Detection